# Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [1]:
import pandas as pd
import numpy as np

In [48]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
results = ratings.groupby('userId').agg({'timestamp': ['min','max'], 'rating': 'size'})
results.rename(columns={'min': 'timestamp_min', 'max': 'timestamp_max', 'size': 'rating_count'}, inplace=True)
results.head()

timestamp                     rating
       timestamp_min timestamp_max rating_count
userId                                         
1          964980499     965719662          232
2         1445714835    1445715340           29
3         1306463323    1306464293           39
4          945078428    1007574542          216
5          847434747     847435337           44

In [4]:
results.columns = results.columns.droplevel()

In [5]:
results.head()

,timestamp_min,timestamp_max,rating_count
userId,,,
1,964980499,965719662,232
2,1445714835,1445715340,29
3,1306463323,1306464293,39
4,945078428,1007574542,216
5,847434747,847435337,44


In [7]:
results = results[results['rating_count'] > 100]
results.head()

,timestamp_min,timestamp_max,rating_count
userId,,,
1,964980499,965719662,232
4,945078428,1007574542,216
6,845553109,845556915,314
7,1106635416,1176181731,152
10,1455301553,1455619275,140


In [16]:
def lifetime(results):
    return results['timestamp_max'] - results['timestamp_min']

results['lifetime'] = lifetime(results)
results.head()

,timestamp_min,timestamp_max,rating_count,lifetime
userId,,,,
1,964980499,965719662,232,739163
4,945078428,1007574542,216,62496114
6,845553109,845556915,314,3806
7,1106635416,1176181731,152,69546315
10,1455301553,1455619275,140,317722


# Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.



In [17]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [18]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [19]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [20]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [40]:
data = rzd.merge(auto, how = 'outer', on = 'client_id')

In [41]:
data = data.merge(air, how = 'outer', on = 'client_id')
data

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [45]:
data.loc[data.rzd_revenue.isnull(), 'rzd_revenue',]=0
data.loc[data.auto_revenue.isnull(), 'auto_revenue',]=0
data.loc[data.air_revenue.isnull(), 'air_revenue',]=0
data

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [47]:
full_data = data.merge(client_base, how = 'outer', on = 'client_id')
full_data

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


# Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

In [66]:
coordinates = pd.DataFrame(
    {      'user_id': [1, 2, 3, 2, 4],
        'visit_id': [111, 112, 113, 114, 115],
        'coordinate': ['55.755831 37.617673', '56.755831 37.619873', '55.755831 38.617673', '67.755831 37.617673', '55.255831 37.017673'],
    }
)
coordinates

,user_id,visit_id,coordinate
0,1,111,55.755831 37.617673
1,2,112,56.755831 37.619873
2,3,113,55.755831 38.617673
3,2,114,67.755831 37.617673
4,4,115,55.255831 37.017673


In [67]:
purchases = pd.DataFrame(
    {      'user_id': [1, 2, 3, 2, 4],
        'visit_id': [111, 112, 113, 114, 115],
        'purchase_id': [201, 202, 203, 204, 205],
    }
)
purchases

,user_id,visit_id,purchase_id
0,1,111,201
1,2,112,202
2,3,113,203
3,2,114,204
4,4,115,205


In [71]:
# Для того что бы идентифицировать координаты необходимо знать информацию о визите конкретного пользователя (visit_id) так как пользователь может зайти из разных мест

cp_merge = purchases.merge(coordinates, how = 'outer', on = 'visit_id')
cp_merge

,user_id_x,visit_id,purchase_id,user_id_y,coordinate
0,1,111,201,1,55.755831 37.617673
1,2,112,202,2,56.755831 37.619873
2,3,113,203,3,55.755831 38.617673
3,2,114,204,2,67.755831 37.617673
4,4,115,205,4,55.255831 37.017673


In [ ]:
# Координаты дают понять локации с наибольшим кол-вом запросов от пользователей; предпочтения покупателей в каких то определенных товарах в опреленных местах и т. д.